# Convert to Pseudobulk using Seurat

In this notebook, we utilize the normalization and aggregation functions in the Seurat package convert our single-cell data to pseudobulk values for each cell type per sample.

## Load packages

In [1]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }

quiet_library(dplyr)
quiet_library(hise)
quiet_library(H5weaver)
quiet_library(purrr)
quiet_library(furrr)
quiet_library(Seurat)

In [2]:
plan(multicore, workers = 12)

In [3]:
min_detection <- 0.1

In [4]:
keep_cohorts <- c("BR1", "BR2")

## Helper functions
This function assists in reading cell metadata data directly from .h5ad files into R

In [5]:
read_h5ad_cell_meta <- function(h5ad_file) 
{
    h5ad_contents <- H5weaver::h5ls(h5ad_file)
    obs_locs <- h5ad_contents$full_name[h5ad_contents$group == "/obs"]
    obs_locs <- obs_locs[!obs_locs %in% c("/obs/__categories", "/obs/_index")]
    obs_locs <- obs_locs[!grepl("Unnamed", obs_locs)]

    h5ad <- H5Fopen(h5ad_file)

    obs_list <- lapply(obs_locs, function(loc) {h5read(h5ad, loc)})

    obs_list <- lapply(
        obs_list,
        function(obs) {
            if(length(obs) == 2) {
                vals <- vector(length = length(obs$codes))
                vals[obs$codes >= 0] <- as.vector(obs$categories)[as.vector(obs$codes + 1)]
                vals[obs$codes == -1] <- NA
            } else {
                vals <- as.vector(obs)
            }

            vals
        }
    )

    obs_list <- lapply(obs_list, as.vector)
    names(obs_list) <- sub(".+/", "", obs_locs)

    H5Fclose(h5ad)
    as.data.frame(obs_list)
}

This function converts from .h5ad expression values to pseudobulk

In [6]:
h5ad_to_pseudobulk_seurat <- function(h5ad_file, group_by, min_detection = 0.1, keep_cohorts) {
    meta <- read_h5ad_cell_meta(h5ad_file)
    meta <- meta %>%
      filter(cohort.cohortGuid %in% keep_cohorts)
    
    meta$pb_group <- meta[[group_by[1]]]
    if(length(group_by) > 1) {
        for (i in 2:length(group_by)) {
            meta$pb_group <- paste(meta$pb_group, meta[[group_by[i]]], sep = ".")
        }
    }
    meta$pb_group <- gsub(" ","-",meta$pb_group)
    meta$pb_group <- gsub("_","-",meta$pb_group)
    
    mat <- read_h5ad_dgCMatrix(h5ad_file, feature_names = "_index")
    
    genes <- rownames(mat)
    keep_genes <- genes[!grepl("^RP|^MT-|^LINC|orf",genes)]
    
    type_meta <- split(meta, meta$AIFI_L3)
    type_mat <- map(type_meta, function(meta) { mat[keep_genes,meta$barcodes] })
    
    type_detection <- map(
        type_mat,
        function(mat) {
            mat <- t(mat)
            diff(mat@p) / nrow(mat)
        }
    )
    
    type_mat <- map2(
        type_mat, type_detection,
        function(type_mat, detection) {
            type_mat[detection >= min_detection,]
        }
    )
    
    type_pb <- map2(
        type_meta, type_mat,
        function(meta, mat) {
            rownames(meta) <- meta$barcodes
            
            so <- CreateSeuratObject(
                counts = mat,
                meta.data = meta
            )
    
            AggregateExpression(
                so,
                assays = "RNA",
                return.seurat = FALSE,
                group.by = "pb_group")$RNA
        }
    )
    
    type_pb
}

In [7]:
h5ad_to_pseudobulk_meta <- function(h5ad_file, group_by, keep_cohorts) {
    meta <- read_h5ad_cell_meta(h5ad_file)
    meta <- meta %>%
      filter(cohort.cohortGuid %in% keep_cohorts)
    
    meta$pb_group <- meta[[group_by[1]]]
    if(length(group_by) > 1) {
        for (i in 2:length(group_by)) {
            meta$pb_group <- paste(meta$pb_group, meta[[group_by[i]]], sep = ".")
        }
    }
    meta$pb_group <- gsub(" ","-",meta$pb_group)
    meta$pb_group <- gsub("_","-",meta$pb_group)
    
    meta <- meta %>%
      select(pb_group, cohort.cohortGuid,
             starts_with("subject"),
             starts_with("sample"),
             starts_with("AIFI_")) %>%
      unique() %>%
      rename(barcodes = pb_group)

    split(meta, meta$AIFI_L3)
}

## Retrieve files to process in HISE

We'll retrieve our clean, non-normalized .h5ad datasets for each AIFI_L2 class from HISE:

In [8]:
h5ad_uuids <- list(
    "ASDC" = "93995785-045f-4068-84b6-2caffadd2046",
    "CD14_monocyte" = "889376f0-2f6d-4eb5-83b1-09a5dbe7bddb",
    "CD16_monocyte" = "fdb1a13e-9f00-458c-87b3-490b0cbe5d60",
    "CD56bright_NK_cell" = "80a1f0a7-b295-4529-b94a-28e5a5cf5d99",
    "CD56dim_NK_cell" = "c737f38d-489a-496d-9543-0df3cd28b73e",
    "CD8aa" = "8eb443ae-8416-4cec-a713-239f3c09a049",
    "cDC1" = "2030a86a-8496-49b9-9648-609963037bf5",
    "cDC2" = "2e083c20-6461-4288-a55c-aa1fc0b0dac9",
    "DN_T_cell" = "b4ba5cd7-af29-4b79-b383-b02d22d6a676",
    "Effector_B_cell" = "79182a91-5c2c-495c-9d9e-211c2c1e7ce0",
    "Erythrocyte" = "394197d7-85f0-46df-89cc-b837a7de107b",
    "gdT" = "b5c5cc8d-e19e-46d6-8313-2901c19911c9",
    "ILC" = "f61c717f-f176-4e19-b886-4418d4585f86",
    "Intermediate_monocyte" = "c6a7c833-27c7-4f5e-8c5b-500d50e6d52d",
    "MAIT" = "2472e2a9-851f-4bfb-a4a0-266f1d307d8e",
    "Memory_B_cell" = "f5deed80-15d6-4faf-9e83-bec55d337656",
    "Memory_CD4_T_cell" = "cc12267d-73e4-4b52-b1de-337d218b31c2",
    "Memory_CD8_T_cell" = "25fc1e17-0dd3-478a-a2c2-a6871b676b3d",
    "Naive_B_cell" = "c57030dd-4c66-4be1-9d06-cc8f505c24d6",
    "Naive_CD4_T_cell" = "bec3a130-218a-4095-9183-b5908872e109",
    "Naive_CD8_T_cell" = "0590000f-c9c6-4b3d-a613-1fa8bc65abbd",
    "pDC" = "21954e50-0e55-484e-aa54-36d892bb52b6",
    "Plasma_cell" = "896280a1-b04d-4f26-9549-80e8c16dc612",
    "Platelet" = "5304bc14-1539-4443-a2dc-99632db631ea",
    "Progenitor_cell" = "054798fb-1d63-4b24-8d0a-ad61f68ebcbc",
    "Proliferating_NK_cell" = "72ec0f6a-0994-4f5f-9c8f-e46b3846a83d",
    "Proliferating_T_cell" = "ac9b7079-dd80-425d-8923-945d900cc445",
    "Transitional_B_cell" = "3a2e0fe0-a199-47f2-bab3-4b2f5d905451",
    "Treg" = "c9c5f690-baaa-46fc-88c7-a723dbc95986"
)

In [9]:
res <- cacheFiles(h5ad_uuids)

In [10]:
h5ad_files <- list.files(paste0("cache/", h5ad_uuids), pattern = ".h5ad", full.names = TRUE)

In [11]:
test <- read_h5ad_cell_meta(h5ad_files[1])

In [12]:
names(test)

[1] "AIFI_L1"                     "AIFI_L2"                    
 [3] "AIFI_L3"                     "barcodes"                   
 [5] "batch_id"                    "cell_name"                  
 [7] "cell_uuid"                   "celltypist.low"             
 [9] "chip_id"                     "cohort.cohortGuid"          
[11] "doublet_score"               "file.id"                    
[13] "hto_barcode"                 "hto_category"               
[15] "keep"                        "leiden"                     
[17] "log1p_n_genes_by_counts"     "log1p_total_counts"         
[19] "log1p_total_counts_mito"     "n_genes"                    
[21] "n_genes_by_counts"           "n_mito_umis"                
[23] "n_reads"                     "n_umis"                     
[25] "original_barcodes"           "pbmc_sample_id"             
[27] "pct_counts_in_top_100_genes" "pct_counts_in_top_200_genes"
[29] "pct_counts_in_top_500_genes" "pct_counts_in_top_50_genes" 
[31] "pct_counts_mito"             "pool_id"                    
[33] "predicted_doublet"           "sample.drawDate"            
[35] "sample.sampleKitGuid"        "sample.visitName"           
[37] "seurat.l1"                   "seurat.l1.score"            
[39] "seurat.l2"                   "seurat.l2.5"                
[41] "seurat.l2.5.score"           "seurat.l2.score"            
[43] "seurat.l3"                   "seurat.l3.score"            
[45] "subject.biologicalSex"       "subject.birthYear"          
[47] "subject.bmi"                 "subject.cmv"                
[49] "subject.ethnicity"           "subject.race"               
[51] "subject.subjectGuid"         "total_counts"               
[53] "total_counts_mito"           "well_id"

## Convert to pseudobulk for each type

In [13]:
group_by <- c("AIFI_L3", "subject.subjectGuid", "sample.visitName")

In [14]:
pb_mats <- future_map(
    h5ad_files,
    h5ad_to_pseudobulk_seurat,
    group_by = group_by,
    min_detection = min_detection,
    keep_cohorts = keep_cohorts
)

In [15]:
pb_mats <- unlist(pb_mats, recursive = FALSE)

In [16]:
pb_meta <- future_map(
    h5ad_files,
    h5ad_to_pseudobulk_meta,
    group_by = group_by,
    keep_cohorts = keep_cohorts
)

In [17]:
pb_meta <- unlist(pb_meta, recursive = FALSE)

## Save to .rds of metadata and matrices for later use in R

In [18]:
pb_data <- list(
    mats = pb_mats,
    meta = pb_meta
)

In [19]:
out_rds <- paste0("output/ref_pbmc_AIFI_L3_seurat_pseudobulk_list_", Sys.Date(), ".rds")
saveRDS(pb_data, out_rds)

## Save to .h5 files for later use via H5weaver

In [20]:
pb_h5_lists <- future_map2(
    pb_mats, pb_meta,
    function(mat, meta) {
        h5_list <- list(
            matrix_dgCMatrix = mat,
            matrix = list(
                observations = as.list(meta)
            )
        )
        h5_list_convert_from_dgCMatrix(h5_list)
    }
)

In [21]:
out_h5 <- map(
    names(pb_h5_lists), 
    function(ct) {
        ct <- gsub(" ", "-", ct)
        paste0("output/ref_pbmc_AIFI_L3_",
               ct,
               "_seurat_pseudobulk_",
               Sys.Date(),
               ".h5")
    }
)

In [22]:
walk2(
    pb_h5_lists, out_h5,
    write_h5_list
)

combine all of these in .tar file for upload

In [23]:
h5_tar <- paste0("output/ref_pbmc_AIFI_L3_seurat_pseudobulk_h5_", Sys.Date(), ".tar.gz")
system_call <- paste(
    "tar -czf", h5_tar, "output/*.h5"
)
system(system_call)

## Upload results to HISE

In [24]:
study_space_uuid <- "64097865-486d-43b3-8f94-74994e0a72e0"
title <- paste("PBMC Ref. Seurat Pseudobulk", Sys.Date())

In [57]:
in_list <- h5ad_uuids

In [58]:
out_list <- list(out_rds, h5_tar)
out_list

[[1]]
[1] "output/ref_pbmc_AIFI_L3_seurat_pseudobulk_list_2024-04-04.rds"

[[2]]
[1] "output/ref_pbmc_AIFI_L3_seurat_pseudobulk_h5_2024-04-04.tar.gz"

In [59]:
uploadFiles(
    files = out_list,
    studySpaceId = study_space_uuid,
    title = title,
    inputFileIds = in_list,
    store = "project",
    doPrompt = FALSE
)

[1] "Authorization token invalid or expired."
[1] "Retrying..."


$files
$files[[1]]
[1] "output/ref_pbmc_AIFI_L3_seurat_pseudobulk_list_2024-04-04.rds"

$files[[2]]
[1] "output/ref_pbmc_AIFI_L3_seurat_pseudobulk_h5_2024-04-04.tar.gz"


$traceId
[1] "17feaa57-c0a8-47c1-a74d-abc54c670fb8"

In [60]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] Seurat_5.0.1       SeuratObject_5.0.1 sp_2.1-2           furrr_0.3.1       
 [5] future_1.33.1      purrr_1.0.2        H5weaver_1.2.0     rhdf5_2.46.1      
 [9] Matrix_1.6-4       data.table_1.15.0  hise_2.16.0        dplyr_1.1.4       

loaded via a namespace (and not attached):
  [1] RColorBrewer_1.1-3  